基于物理原理的原子尺度性质机器学习

**摘要**:

我们简要总结了最近在材料建模中使用的核回归方法,以拟合函数,特别是势能面,并强调线性代数框架如何用于预测和训练势能的线性函数,例如总能量和原子力.
然后,我们详细介绍原子位置(SOAP)描述符和内核的平滑重叠,显示它是如何从平滑原子密度的抽象表示以及它如何与几种流行的基于密度的原子结构描述符相关联的.
我们还讨论了最近的概括,它们可以精确控制不同原子种类之间的相关性,预测和拟合张量性质,以及如何构建结构核 - 适用于比较整个分子或周期系统 - 超越局部环境的加性组合.

# 简介

在过去几年中，将机器学习方法应用于材料和分子模型问题的活动激增，这些问题主要得益于这些技术在可以被称为人工智能的过程中取得的明显成功。这些成功源于科学界在填充大量数据时所获得的集体经验，这些数据具有非常复杂的功能形式，涉及大量自由参数，同时仍然保持对规律性的控制，从而避免灾难性的过度拟合。在分子建模的背景下，潜在能量表面的经验拟合当然已经使用了数十年。实际上,人们普遍认为这是在长时间尺度上模拟非常大的系统(数千个原子)的唯一实用方法(数百万个时间步长).

传统上，当拟合原子相互作用的经验模型时，通过编写以一维函数表示的函数形式来确保规律性，例如，对电势，球对称原子电子密度，键序（作为邻居数的函数）这些功能很容易在视觉上检查，以确保它们具有物理和化学意义，例如，对电位在远距离处变为零，并且在近距离时强烈排斥，原子电子密度随着距离而减小，即电子密度嵌入函数是凸的等等。此外，这些自然属性易于构建成一维函数形式或在参数优化中作为约束强制执行。人们普遍认为,采用这种"物理上有意义的"功能形式是实现经验模型良好可转移性的关键

In [13]:
ggtrans("""
It is widely held that employing such “physically meaningful” functional forms is the key to achieving good transferability of the empirical models
""")

人们普遍认为,采用这种"物理上有意义的"功能形式是实现经验模型良好可转移性的关键


然而，还认识到，可以从这些一维函数构建的有限函数形式最终限制了这些经验模型可以实现的精度。在尝试使用更灵活的功能形式通过高维度替换它们时，必须立即改变两件事。第一个是目标数据。当只拟合几个参数时，很自然地要求被认为是被解决的科学问题的核心的重要可观察量被正确地再现，并且如果它们是拟合的一部分，例如熔点，则最容易做到这一点。然而，在参数非常多的情况下，它们的优化也需要大量的评估，并且使用复杂的可观测量作为目标变得不切实际。因此，科学上相关的观察也是如此。因此，通过选择的电子结构方法计算，作为势能面的目标数据值是很自然的。然后，可以通过大量计算的输入数据容易地平衡大量的自由参数。

必须改变的第二件事是如何控制潜在的物理意义的潜在行为。检查手动高维函数以确保其预测对于所有可能的配置在物理和化学上都是有意义的是不实际的。因此，尽可能多地将关于限制行为和规律性的先验信息构建到功能形式中变得更加重要（关于我们感兴趣的平滑度的技术词）。回顾最近的工作，本文提出了一个如何做到这一点的示例框架。关键目标是创建泛函形式，以保持
* i. 属性相对于相似原子的排列的不变性
* ii.标量的不变性和具有三维旋转的张量性质的协方差，以及
* iii. 相关性的连续性和规律性改变原子坐标，包括通过包含有限截止值来紧密支持原子相互作用。

越来越多的证据表明，严格执行这些具有物理动机的属性是非常有益的，并且几个最成功的原子级系统机器学习方案是围绕对称参数构建的。一种可能的方法是根据内部坐标来描述系统，该内部坐标满足自动旋转不变性，然后明确地对称描述符的向量或描述符与属性之间的函数关系。置换不变多项式是一个非常有效地模拟小分子势能面的例子（参见，例如，Braams和Bowman 2009的工作））。根据原子间距离对描述符进行排序也被用作以引入衍生不连续性为代价获得置换不变性的方法（Rupp等人2012 ; Faber等人2015 ; Zhang等人2018）。我们将在本文中关注的另一种可能性是从每个结构的原子密度表示开始 - 这对于原子排列是自然不变的 - 然后构建一个对平移和旋转进一步不变的描述符。

无论哪种方式，一旦获得了对每个结构的适当描述，就可以在回归方案的水平上实现进一步的正则化。为此，两个突出的技术是使用人工神经网络和核岭回归（Bishop 2016）。我们在这里使用后者的形式主义，关于这些技术的许多更多细节可以在本卷的其余部分找到。内核方法从内核函数的定义开始，该函数将与一组代表性原子配置组合以构造拟合的基函数。它是一个标量函数 - 至少在学习标量时 - 有两个输入参数，在本例中是两个原子结构。它的值应该量化由它的两个参数表示的原子配置的相似性，并且它可以（但不必）从它们的相关描述符开始定义。当两个参数相等（或等效于对称操作）时，该值应该是最大的，对于最大不同的配置，该值应该是最小的。内核在改变原子配置时能够捕获函数变化的程度将决定拟合的效率。对应性越好，实现给定的拟合精度所需的基函数越少。

# 核拟合

我们首先简要介绍内核回归拟合方法，更多细节见Bishop（2016），Rasmussen和Williams（2006），以及Schölkopf和Smola（2002）。在原子结构上定义的函数表示为内核基函数的线性组合(和)

$ 
\displaystyle 
\begin{aligned} 
f (\mathcal{A})= \sum_{{\mathcal{B}} \in M} x_{\mathcal{B}} K(\mathcal{A},\mathcal{B}),{} 
\end{aligned}
$

In [14]:
youdao2("""A function defined on an atomic structure is represented as a linear sum over kernel basis functions,""")

在原子结构上定义的函数表示为核基函数的线性和，


1. 其中总和在代表性的一组配置M上运行，从输入配置的总集合N中选择。组合成矢量x的系数集合通过求解当可用数据（例如，针对一组结构评估的目标函数的值）被替换为等式1时获得的线性系统来确定。（1）。在最简单的情况下，有一个输入数据值对应于每个原子配置。设y为所有可用输入数据的向量，K为内核矩阵，其行和列对应于原子结构，因此K的元素对于结构和分别对应的行和列是。然后通过求解最小二乘意义上的线性系统来获得拟合，即最小化二次损失函数：

$\displaystyle \begin{aligned} \ell{}(\mathbf{x}) = \|\mathbf{K} \mathbf{x} - \mathbf{y}\|{}^2 . \end{aligned}$

2. 教科书案例是当我们具有可用目标数据的所有配置的集合作为代表集合使用时（即，N  =  M），K是正方形，并且只要它是可逆的，最佳解决方案是：
x = K.- 1y。
3. 实际上，对于大型数据集，不必使用数据集中的所有配置作为代表。在这种情况下，中号  ⊂  Ñ，溶液是由伪逆给出
X中号= （K中号ñķñ中号）- 1ķ中号ñÿñ，
4. 在这里我们使用下标来强调的是，向量元素对应于，例如，该组ÿ  ≡ ý Ñ是具有用于每个输入数据结构和一个元件中的数据矢量X  ≡ X 中号是系数的矢量，每个代表性配置。内核矩阵上的下标表示数组切片，即是矩形矩阵，其元素对应于代表性配置和输入配置之间的内核值。  ķ中号ñ= K.⊤ñ中号 
使用远小于结构总数的代表集在计算成本方面具有显着优势，通常不会降低拟合精度。训练成本主要通过计算伪逆而得到，伪逆是O（NM 2），其在训练数据N的大小上是线性的，将模型尺度评估为O（M），现在与训练的大小无关数据。这些成本标度类似于具有固定数量节点的人工神经网络。

虽然上述解决方案在形式上是正确的，但人们普遍认为它们会导致数值不稳定和过度拟合，即，它们是试图最大化输入数据拟合的解决方案，即使这可能是不可取的，这几乎总是如此。乍一看，这可能听起来令人惊讶，因为电子结构计算可以确定性，在其参数方面具有精确的收敛行为，例如k点采样，SCF容差等。但是，实际计算永远不会收敛到机器精度并且，不同配置的势能值之间产生的不一致性不是传播到拟合的势能面所希望的。在进行拟合之前，可以容易地评估这种不一致的程度。以前的经验（Szlachta等人2014 ; Dragoni等人2018年）表明，在用于材料应用的大型数据库中，由于相关的计算成本，由于k点采样引起的误差是主要的并且难以降低到低于meV /原子。

如果我们正在拟合一个潜在的能量表面，其表示不能完全表征整个系统的原子位置，例如，由于有限的截止，或者为获得计算效率而做出的其他一些选择，预计拟合不会是确切地说，与输入数据量无关。有时，这样的模型误差。也可以通过测量由其它原子超出截止的原子（Bernstein等人的力的贡献评估先验，例如，在一个有限截止的情况下，2009年事务所和Csányi; 2017 ;藤挂等人，2018年）。

这两个考虑因素表明，在线性系统中允许一些“松散”可能是有益的，因为它可以被利用以允许更小的线性系数，使得拟合更加规则并因此更好地进行外推。我们收集在N  ×  N矩阵的对角线上的每个目标数据值的拟合中我们期望的误差Λ。规范问题的常用程序是由Tikhonov等人提出的。（1995年）。具体来说，在“核岭回归”（和等效的“高斯过程回归”，相同的贝叶斯视图）中，Tikhonov矩阵被选择为M个代表点之间的核矩阵，K MM。通过高度规则（“平滑”）的核函数，这种正则化导致平滑拟合，并且Λ元素的大小控制拟合精度和平滑度之间的权衡。相应的解决方案是
x =（ K +Λ）- 1y。
5. 对于方形问题，和
X中号= （K中号中号+ K.中号ñΛ- 1ķñ中号）- 1ķ中号ñΛ- 1ÿñ，
6. 对于矩形问题，我们再次强调了索引集。该解决方案相当于最小化
∥ ķ X - ÿ ∥2Λ- 1+ ∥ X ∥2ķ，
7. 这表明Λ中的容差的倒数等于不同数据点上的权重。
通过线性系统的解决方案，新结构的拟合函数的值可写为  C 
F（C.）= KC中号X中号。
8. 请注意切片只是一个向量，内核在新结构之间给出了元素和代表集M中的结构。  ķC中号   C 

## 选择代表集

接下来，我们将介绍一些选择代表性环境的方法，在这些环境中，方程式中的和。（1）被采取。这可以通过简单的随机采样来完成，但我们发现利用这种自由来优化插值精度是有利的。为此提出了许多策略（Hartigan和Wong 1979 ; Prabhakaran等人，2012年）），我们讨论在这种情况下成功使用的两个。对此的一种方法是最大化代表集的元素之间的不相似性。用于选择代表集的配置的贪婪算法是“最远点采样”，其中我们从随机选择的结构开始，然后迭代地选择下一个结构，即距离集合中已有的任何结构最远的结构。 （Gonzalez 1985 ; Ceriotti等人2013 ;Bartók等人2017）。两个结构之间的距离是在“内核度量”中测量的，定义为（Schölkopf和Smola 2002）：
d2（A.，B）= K（A.，A）+ K.（B，B）- 2 K.（A.，B）。
（9）
该算法用于回归任务选择的分子，从而使数据集大小的显著减少对精度的给定水平的表现良好（德等人。2016）。
已成功使用的另一种技术基于矩阵分解，当核函数是线性或描述符向量的低阶多项式时，这尤其具有吸引力。考虑描述符向量的矩阵，d，其中每一行是一个输入原子结构的描述符向量，使得线性核是ķ  =  DD ⊤。我们希望选择比总数少很多的行，这些行跨越所有行跨越的空间。这是矩阵表示的问题，具体地说，代表集应该用作K和/或D的低秩近似。对此的一种解决方案称为CUR矩阵分解（Mahoney和Drineas2009），可以应用于 K或 D，如果描述符向量的长度小于数据点的数量，则后者便宜得多。

为了确定最佳代表性配置集，我们从D的奇异值分解开始，
D =U.小号V⊤。
（10）
对于每个数据点，计算杠杆分数，基本上是顶部奇异向量对该配置的权重。
π一个= 1ķΣξ= 1ķ（你ξ一个）2
（11）
其中是ξ- th左奇异向量的元素，对应于结构。总和在第一个k奇异向量上运行，例如，k  = 20是典型的。配置包含在代表集中，其概率与其杠杆分数成比例，。确定性变体是一次选择一个结构 - 具有最高杠杆得分的结构 - 从描述符矩阵中删除关联的行，并将D的剩余行正交化  üξ一个   一个   一个   π一个   一个 相对于它。然后可以选择下一个数据点，在现在较小的描述符矩阵上重复相同的过程（Imbalzano等人，2018）。
请注意，在高斯过程文献中，使用数据子集构建基础称为稀疏化（Quinonero-Candela和Rasmussen 2005 ; Snelson和Ghahramani 2005），即使近似依赖于低秩矩阵重建而不是核矩阵稀疏。

## 内核的线性组合

当拟合材料的原子间势时，为原子能量构建模型有时称为“场地能量”。这既是为了提高计算效率又是为了降低结构和属性之间功能关系的复杂性：每个原子能量只是对应于坐标的有限自由度的函数。相邻原子，因此可以独立于任何其他原子能量进行评估。实际上，这是原子间势的定义特征，与明确包括离域电子的量子力学模型形成对比。从原子能量到总能量是微不足道的，后者是前者的总和。然而，走向另一个方向并不是明确的。总能量可以从量子力学模型计算，但原子能量没有唯一定义，并且找到最佳可能的总能量分离成原子能量成为拟合任务的一部分。在同一基础上处理这两个转换有助于。假设我们想要预测两个（或更多）配置的函数值之和。对于结构的两个能量之和的简单情况  一个 和，预测只是个别函数值预测的总和，例如：  乙 
Ë吨ö 吨= E.（A.）+ E.（B）= K一个中号X中号+ K.乙中号X中号。 
（12）
如果我们将新的“和核”定义为多个新配置和代表集之间的核值之和，则上述总能量预测的表达式采用与各个函数值的预测相同的形式。对于一些设置我的新配置，让
ΣKM=∑A∈IKAM,
（13）
其中是sum-kernel值的向量，其中每个元素都是I和a中所有配置之间的内核之和在代表集M中给出配置。在配置的预测的总能量我然后  ΣKM 
Etot=ΣKMxM.
（14）
可以使用相同的求和内核来拟合模型以求和数据，而不是单个函数值。这对于拟合材料系统的原子间势是至关重要的，因为只有总能量，而不是原子能本身，可以从电子结构计算中获得。同时，为了在原子间势能中实施有限的短程，我们必须将这种潜力表达为原子能。使用sum-kernel，这很简单，可以保留（1）中的原始函数形式，然后我们现在最小化：
∥ΣKx−Etot∥2,
（15）
其中Σ ķ是包含在输入数据库和代表组和所述矢量的所有配置的总和内核值的矩阵ê TOT是对应的总能量数据的收集。

## 导数

方程的显式解析函数形式。（1）导致关于原子坐标的分析导数，例如，在拟合能量的情况下的力。考虑到目前我们计算以原子为中心的数量的导数的简单情况，我们定义作为结构中所有原子坐标的导数矢量。我们使用符号来表示适用于内核的第一个参数的派生运算符和  f(A)   ∇A   A   ∇←   ∇→ 表示适用于第二个参数的衍生物。的衍生物是非零的仅针对属于该结构的原子，并且然后由微分等式给出。（1）：  f(A)   A 
∇Af(A)=∑B∈MxB∇←AK(A,B)=K∇AMxM,
（16）
我们在那里引入了符号来指示包含内核衍生物的矩阵相对于所有相关的原子坐标。与上述总和的情况类似，核函数的梯度也可用于拟合模型而不是目标值，而是梯度数据（Solak等人，2003）。当目标代表潜在的能量表面时，这尤其有用。当使用典型的电子结构的方法，相对于全部原子的位置计算梯度的成本仅比计算的能量，但产量更多的信息，3个的成本多一点Ñ段数据用于Ñ  ķ∇ 一中号  - 原子结构。可以采用两种方法来合并梯度信息。在第一个，用于Bartók等人。（2010）和该组的后续工作（巴托克等人2013a ;巴托克等人2017 ;什拉赫塔等人2014 ;巴托克和Csányi 2015 ;事务所和Csányi 2017 ; John和Csányi 2017 ;藤挂等人2018 ;德拉戈尼等人2018 ; Deringer等人2018 ; Caro等人2018 ; Rowe等人2018 ; Nguyen等人2018），能量的功能形式再次保持与方程式相同。（1）。相应的损失函数是
∥ ķ∇ ñ中号X中号- ÿ∇ ñ∥2，
（17）
其中ý ＆dtri Ñ指梯度对在所述一组输入结构和的所有原子的级联矢量ķ ∇ NM到内核衍生物的相应的矩阵。对该系数的溶液的形式从不变（5）或（6）与ķ ∇ NM服用的作用ķ NM。
在第二种方法中，最近在Chmiela等人中使用过。（2017），内核的衍生物是拟合的函数形式的基函数，
F（A.）= Σ乙∈ 中号X∇ 乙＆CenterDot;＆∇→乙ķ（A.，B），
（18）
其中包含相对于结构中原子的每个导数的一个权重。对于n原子结构，基函数和相应系数的数量现在要大得多，n nM。由于模型适用于衍生物，由（18）的梯度给出，损失是  X∇ 乙   乙 
∥ ķ∇ ñ∇ 中号X∇ 中号- ÿ∇ ñ∥2，
（19）
目标属性可以计算为
F（A.）= K一个∇ 中号X∇ 中号，
（20）
和它们的衍生物
∇一个F（A.）= K∇ 一∇ 中号X∇ 中号。
（21）
从方程式可以看出这种方法的最初动机。（19）其中矩阵可以直接理解为原子力之间的核（并且在M  =  N的情况下，在输入数据力之间）。
两种方法都构成了从表示其梯度的数据学习函数的有效方法，仅在核心基础的选择上有所不同。内核导数基函数也可以与简化代表集一起使用，目前尚不清楚哪种方法更好或实际上是组合：可以根据数量和数量选择不同的基函数（内核或其衍生物）。可用的数据类型以及代表集的大小和选择。

## 从线性功能学习

我们可以自然地组合求和核和导数核，并为（1）中的系数写出单个最小二乘问题，该问题被解决以适应所有可用总能量，力和维里应力数据的原子间势（唯一的条件）因为输入数据必须可表达为对函数值的线性运算。我们将y定义为包含所有输入数据的L个分量的向量：训练数据库中的所有总能量，力和维里应力分量，y '为具有N个分量的向量，其中N个分量包含N个原子环境中的未知原子能量。数据库，和 在新窗口中打开图像 作为大小为L × N的线性微分算子，它将y与y'连接，使得 在新窗口 中 打开图像 （请注意，我们在这里使用的 新窗口中 的 打开图像 的定义是Bartók中的转置和Csányi 2015年）。最小二乘问题现在是最小化：
在新窗口中打开图像
（22）
并且系数的表达式由下式给出：
在新窗口中打开图像
（23）
有益的是，当系统只包含两个原子的写下上述矩阵的简单情况甲和乙，与位置矢量ř 甲，- [R 乙，目标总能量ë，和目标力˚F 甲  ≡（˚F 斧，˚F Ay的，˚F 了Az）和˚F 乙  ≡（˚F Bx的，˚F 通过，˚F 的Bz）。然后给出数据向量
y =[ EF一个xF一个ÿF一个zFB xFB yFB z]⊤。
（24）
拟合的目的是确定两个未知原子能函数ε 甲和ε 乙作为两个原子为中心的原子的环境中，的函数和分别是。的总能量是它们的总和，ê  =  ε 甲  +  ε 乙，和力需要包括交叉项，  一个   乙 
F一个F乙= ∂ε一个∂[R一个+ ∂ε乙∂[R一个，= ∂ε一个∂[R乙+ ∂ε乙∂[R乙。
（25）
在这种情况下，代表集由相同的两个原子组成，因此N  =  M，核矩阵是正方形，
K = [ K（A.，A）ķ（B，A）ķ（A.，B）ķ（B，B）]，
（26）
线性算子 在新窗口中打开图像 是一个7×2矩阵，由下式给出
在新窗口中打开图像
（27）
所以将 新窗口 矩阵中的 Open图像 替换为Eq。（23）是
在新窗口中打开图像
（28）
请注意，或不为零，因为原子A存在于环境中原子B，所以，还有明确依赖于r A.  ∇←[R一个ķ（B，B）   ∇←[R一个ķ（A.，B）   乙   ķ（B，A）   ķ（B，B） 
使用Chmiela等人的方法。（2017）对于二聚体，核矩阵是6×6，由下式给出：
ķ∇ 一∇ 乙= ⎡⎣⎢∇←[R一个∇→[R一个ķ（A.，A）∇←[R乙∇→[R一个ķ（B，A）∇←[R一个∇→[R乙ķ（A.，B）∇←[R乙∇→[R乙ķ（B，B）⎤⎦⎥。
（29）
在实践中，总是值得使用所有可用数据，即使一旦拟合收敛于无限量数据的极限，衍生物（力）的信息与能量相同。然而，对于有限数量的数据，通过对角线规则器选择对应于能量和力的权重允许控制拟合，在其再现能量和力的相对精确度方面。

# 基于密度的表示和内核

总结了可用于使用原子级属性及其衍生物作为输入执行内核岭回归的算法，我们现在继续描述用于定义基于物理的本地原子环境描述符的框架以及从它们构建的内核。在kernel ridge回归中，描述符不一定需要明确表达，但也可以通过与标量对应的核函数隐式定义。描述符向量的乘积，跨越（可能无限维）希尔伯特空间（Schölkopf和Smola 2002）。矢量 在新窗口中打开图像   ķ（A.，B）   在这个“再生内核Hilbert空间”确实对应于原子结构，即使可以在没有明确确定向量的情况下计算内核，也可以 在新窗口中 正式编写 Open图像 。

受过量子力学训练的读者将一方面识别描述符和状态向量之间的同构，另一方面识别内核和期望值。这个比喻表明，使用模仿狄拉克符号的形式来制定原子尺度描述符可能是有益的。而在量子力学环境中，问题的物理对称性被建立在哈密顿量中，在机器学习环境中，它们更方便地包含在表示本身中，应该使基本对称性不变，例如原子标记，刚性平移和旋转。在本节中，我们将展示如何从这些直觉开始，可以建立一个对物理对称性自然不变的分子结构的非常抽象的描述，

通过将结构分解为局部环境的集合并通过SO（3）组上的显式对称化，可以包括平移和旋转对称性。这种结构让人想起已经应用于固体对象比较的方法（Kazhdan等人，2003），并自然地导致SOAP描述符和内核（Bartók等人，2013b）以及其他几种基于密度的描述符的流行选择。 -从Behler-帕里内洛对称函数（Behler和帕里内洛2007），以体素密度的描述符（朱音等人2017年的对相关函数的合并（舒特等人）。2014） - 可以视为相同平滑原子振幅的不同投影。SOAP框架的一个特点是它允许人们显式地计算其标量产品构成内核函数的描述符，这允许人们在内核和描述符语言之间来回传递。然后可以修改原子环境描述符以生成非线性内核，以及组合成全局结构内核。我们将简要讨论后者的不同可能方法，或者通过局部描述符的简单线性组合，或者通过考虑被比较的两个结构中的环境对之间的最有效匹配的更复杂的过程。

## 结构描述符的Dirac表示法

让我们引入一个抽象的符号来描述原子结构的组成原子的位置和化学性质。从狄拉克符号中获取量子力学状态的灵感，我们将 新窗口中的开放图像 与每个配置相关联。让我们从一个简单的例子开始，看看如何引入和使用这种形式主义。与量子态的情况非常相似，我们可以根据位置和化学种类来定义与结构相关的酮的具体表示，例如：
在新窗口中打开图像
（30）
其中每个原子的位置由平滑密度g i（原则上可能取决于核电荷和原子i的位置）表示，而 新窗口 中的 开放图像 包含每个原子核电荷的信息。Dirac表示法自然适用于结构之间重叠核的定义， 在新窗口中打开图像 。为了计算这样的积分，可以使用位置表示并假设与不同元素相关联的kets是正交的：
在新窗口中打开图像
（31）
这种基于密度的表示本身并不是非常有用，因为内核对于结构的相对旋转不是不变的，甚至对于空间中的两个结构的绝对位置或它们的周期性表示也不是不变的。然而，它可以作为引入许多最成功的特征描述符的起点，这些特征描述符近年来用于材料和分子的机器学习。为了了解如何，可以从线性缩放电子结构方法和电子物质的近视原理中获取灵感（Yang 1991 ; Galli和Parrinello 1992 ; Goedecker 1999 ; Prodan and Kohn 2005）。然后，我们将注意力从完整结构的描述转移到球形原子环境的描述，人们可以方便地将其放在每个原子的顶部。这也与前面部分讨论的原子中心电位一致，作为该框架的一个明显应用。我们将 在新窗口中 使用符号 Open图像 来指示以结构中第j个原子为中心的环境，并在位置表示中表示为：
在新窗口中打开图像
（32）
其中f c（r ij）是截止函数，其为了计算效率和/或密度信息的定位而将环境限制在以原子为中心的球形区域。以原子为中心的平滑函数通常被认为是均匀宽度的高斯函数，但是很容易将表达式推广到包括对原子种类的依赖性和/或原子与环境中心的距离，这可能是用于例如降低环境周边的描述符的分辨率或使平滑长度尺度适应每个原子种类。
注意，人们还可以将来自相同物种的原子的密度贡献组合成物种依赖的原子振幅，
在新窗口中打开图像
（33）
然后写
在新窗口中打开图像
（34）
这种表示法非常有用，可以揭示不同的描述符如何被视为同一抽象派的替代表示。例如，可以在正交径向函数R n（r）和球谐函数中扩展原子密度。这种扩展中的系数可以写成
在新窗口中打开图像
（35）
另一个例子是，Behler-Parrinello以原子为中心的对称函数已被用于构建基于人工神经网络的材料的原子间势能（Behler和Parrinello 2007 ; Eshet等人2012 ; Morawietz等人2016 ; Cheng等人。2016）和分子（Smith等人2017）可以通过设置基本函数是三角形分布被写入克IJ（[R  - - [R IJ）=  δ（[R  - - [R IJ）和平均与适当的一对加权的原子密度功能G 2，例如：
在新窗口中打开图像
（36）
谱相邻分析势的基函数（Thompson等，2015）也以相同的密度开始，并将其扩展为Bartók等人提出的超球谐函数。（2010）。

## 原子位置的平滑重叠

很明显，基于密度的表示，如Eq。（32）对于整个结构的平移是不变的，但不是对将改变原子邻居振幅的方向的旋转不变。这反映了以下事实： 新窗口中打开图像 形式的标量产品取决于所比较环境的相对方向。在原子位置（SOAP）框架的平滑重叠中，我们使用旋转组的Haar积分（Haar 1933）定义重叠内核的对称版本：
在新窗口中打开图像
（37）
其中积分在所有可能的旋转矩阵上执行。如果基本核被提升到ν次幂，则平均值保留关于原子之间相关性的信息，直到（ν  + 1）次序（Glielmo等人，2018）。正如我们将在下面展示的那样，SOAP框架的一个关键特性是可以给出对称描述符向量的显式表达式 在新窗口中打开图像 ，这对于ν  = 1,2 非常容易管理，但对于高ν变得越来越麻烦。对高阶相互作用的有效描述不会过多地增加分析评价的复杂性（37）可以通过操纵ν  = 2内核来获得，例如，通过采用它的非线性函数。在实践中，已经发现将其提高到功率ζ并将其标准化为1：
在新窗口中打开图像
（38）
足以在最终内核中包含多体贡献。
使用狄拉克符号，很容易看出如何在ν  = 1,2 的情况下给出SO（3）对称化钳的显式表示。 在新窗口中 使用 开放图像 的球谐函数展开，它非常通过引入与旋转相关联的Wigner矩阵，可以自然地分析地执行旋转平均： 在新窗口中打开图像     
在新窗口中打开图像
（39）
由于Wigner矩阵的特性，这大大简化了。只有l  = 0的项存活，这使得可以 根据球谐函数系数明确写出ν = 1对称SOAP描述符
在新窗口中打开图像
（40）
它对应于简单的内核
在新窗口中打开图像
（41）
ν  = 1描述符的位置表示 在新窗口 中 打开图像 自然地产生 新窗口中 的 打开图像 的旋转平均值。这可以通过在位置基础上表达来看出  ķ（1 ）（X.Ĵ，Xķ） 
在新窗口中打开图像
（42）
我们在哪里定义了旋转平均原子密度：
在新窗口中打开图像
（43）
因此，它与标记原子周围的对相关函数密切相关。类似的描述符已被用于分子和材料的机器学习（Schütt等人，2014 ; Faber等人，2015），再次揭示了不同基于原子密度的描述符之间的亲密关系。
在ν  = 1点的描述符集成了所有角度的相关性，并因此不提供环境的唯一表示。具有ν  = 2 的描述符提供关于三体相关的信息，并且还可以以封闭形式相对容易地获得。Haar积分现在包含两个Wigner矩阵的乘积。人们开始利用他们的正交关系
在新窗口中打开图像
（44）
其中ν  = 2对称SOAP描述符读取
在新窗口中打开图像
（45）
该符号对应于Bartók等人引入的功率谱分量。（2013b）和De等人。（2016）， 在新窗口中打开图像 。还要注意，虽然就nlm扩展而言对称化kets的表示非常方便，但它不是唯一的可能性。与Eq相似。（43），可以 在新窗口 中获得 开放图像 的显式位置表示，其提供三体旋转不变相关的完整表示。Behler-Parrinello类型的三体对称函数可以看作是这种表示的投影，类似于方程式中的两体函数的情形。（36）。
ν  = 3 的情况导致了与环境双谱成比例的确切表示（Bartók等人2013b）
在新窗口中打开图像
（46）
虽然这个描述符的维度使得它不切实际，除非以某种方式被稀疏化，它确实可以直接访问更高阶的相关性。一个有趣的细节是， 在新窗口 中 打开图像 ，与ν  = 1,2，情况相反，对镜像对称不是不变的，这使得它能够区分对映体。
最后，应该注意内核Eq的规范化。（38）可以通过规范化SOAP向量来实现，从而可以实现规范化描述符的显式表示。虽然原则上可以写出一个显式表示，产生ζ  > 1 的内核，但它将包含指数增加的项数。与在 新窗口 中 打开图像 的情况一样，这仅在与稀疏化过程结合时才有意义。

## -SOAP：对称性适应高斯过程回归

当为张力性质T建立机器学习模型时，应该考虑目标在对称操作（例如，旋转）的作用下不是不变的，而是协变地变换。在模型中编码适当的协方差属性的最有效策略涉及将张量分解为其不可约的球形分量，即张量元素的组合，其变换为λ阶球谐函数（Varshalovich等人，1988） 。对于这些不可缩减的组件，
Tλμ(R^Xj)=∑μ′Dλμμ′(R^)Tλμ′(Xj)
（47）
如Glielmo等人所示。（2017）关于载体和Grisafi等人的案例。（2018），用于任意阶的张量，人们必须考虑的矩阵值的内核，它描述的不同组件之间的几何关系Ť λ，这可以通过包括一个附加的Wigner矩阵来获得在哈尔积分中的  dλμ μ“（R^） 
在新窗口中打开图像
（48）
对于ν  = 2 的情况，可以基于原子密度的αnlm展开明确地写出对称化的kets：
在新窗口中打开图像
（49）
我们写Eq。（49）以这种形式，因为它有点对称，但CG系数的属性要求m'  =  m  +  μ，因此可以用单个和来评估表达式。此外，每当，表达式的计算结果为0 ，这减少了必须评估和存储的元素数量，并清楚地表达了Eq。（49）减少到标量SOAP Eq。（45）当λ  = 0时。  | l - l“| <λ 
当使用线性模型时，每个对称适应描述符Eq。（49）可用于表示变换为张量分量。在这种情况下，线性度对于保持λ- SOAP 的对称性是必要的。然而，非线性模型可以通过 在 一些λ  = 0描述符的（原则上不同的）非线性函数中 在新窗口中 缩放每个 Open图像 来获得。在内核语言中，可以使用类似于（38）的表达式引入λ- SOAP内核的高阶版本：  ÿλμ   
在新窗口中打开图像
（50）
其中表示Frobenius规范， 新窗口 中的 Open图像 是（标量）SOAP内核。第二项使整个内核非线性，而不影响整个张量内核的对称性。  ∥ ＆CenterDot;＆∥F   

## 内核运算符和描述符优化

Provided one takes a long-range environmental cutoff and chooses a kernel that can represent high orders of many-body interactions, a density-based representation of atomic structures should provide a complete description of any atomic structure and – given a sufficiently complete training set – predict any atomistic property with arbitrary accuracy. In practice, obviously, the accuracy of a model depends on the details of the representation, which is why different descriptors or kernels provide different levels of accuracy for the same training and test set (Faber et al. 2017). The performance of a set of descriptors can be improved by modifying them so that they represent more efficiently the relations between structure and properties.

结构，属性和内核之间的关系可以编码在Hermitian运算符括在平移不变的环境kets之间：  w ^^ 
在新窗口中打开图像
（51）
当内核在旋转上取平均值以获取SOAP内核和相关描述符时，
在新窗口中打开图像
（52）
很明显，如果我们希望能够明确地编写“运算符自适应的” SO（3）描述符，则运算符必须与循环进行通信。这个附加要求意味着 在新窗口的 基础上以 开放图像 编写的的最一般形式是  w ^^   w ^^   
在新窗口中打开图像
（53）
例如，该运算符可以用于将密度作为距离的函数来缩放，或者引入“炼金术内核”，其耦合与不同元素相关联的描述符，并且原则上实现SOAP描述符的更微妙的调整，混合了径向，角度和化学通道，以更有效地表示复杂的化学环境。

## 高效地计算SOAP描述符

标量和张量ν  = 2 SOAP描述符的实际计算 在新窗口中打开图像 需要评估扩展系数 在新窗口中打开图像 。让我们从位置表示中写入的原子密度开始，根据方程式。（35），并考虑将写成宽度为σ的球形高斯函数叠加的情况α型原子的位置。然后，方程式中的球面谐波投影。（35）可以进行分析，导致：  ψαX（r） 
在新窗口中打开图像
（54）
其中总和运行在类型的所有相邻原子α和ι 升指示第一类型的改性球状贝塞尔函数。选择正交径向基函数R n（r）的形式是方便的，这使得可以分析地执行径向积分。
一种可能的选择首先使用高斯型轨道作为非正交基本函数  [R〜ķ（r） 
[R〜ķ（r ）= N.ķ [RķEXP- 12（rσķ）2，
（55）
其中是一个归一化因子，例如。所述一组高斯宽度{的σ ķ }可以选择有效地跨越参与环境定义中的径向间隔。假设平滑切断功能在距离接近1 - [R 切  -  ΔR 切，人们可以采取，让被在等距隔开的位置处达到峰值范围为0和之间的功能- [R 切  -  ΔR 切。  ñķ   ∫∞0d[R [R2[R〜2ķ（r ）= 1   σķ= （r切- δ[R切）max （k- - √，1 ）/ n最大 
While the   R~k(r)  are not themselves orthogonal, they can be used to write orthogonal basis functions   Rn(r)=∑kS−1/2nkR~k(r) , where the overlap matrix   Skk′=∫drr2R~k(r)R~k′(r)  can be computed analytically. The full decomposition of the translationally invariant environmental ket can then be obtained without recourse to numerical integration.

一旦获得原子密度的球形分解，就可以组合系数以给出阶数1和2的SOAP描述符。特别是在存在许多不同的化学物质的情况下，组分的数量可以变得巨大。为简单起见忽略一些对称性，以及如果所有物种都不出现在每个环境中的事实，可以存储描述符的稀疏表示，功率谱包含多个组件的顺序，可轻松达到成千上万。在张量λ- SOAP 的情况下，数量进一步增加到  n2speciesn2maxlmax   λ2n2speciesn2maxlmax 。然而，没有必要计算和存储所有这些描述符：它们中的每一个或任何线性组合是环境的球形不变（协变）描述，并且可以单独用作描述符。这可以被利用来显着降低SOAP计算的计算成本和内存占用，确定描述符的低秩近似。人们可以使用类似于Sect中讨论的降维技术。2.1确定最合适的参考结构。如Imbalzano等人所示。（2018年），CUR分解和基于最远点采样的贪婪选择策略使得有可能将化学精确度预测小有机分子能量所需的SOAP描述符数量减少95％以上。

## 回到结构

每当人们对计算与单个原子相关的属性（例如，他们的核磁共振化学屏蔽或力）感兴趣时，可以直接使用对应于每个环境或两个环境之间的内核的描述符，作为线性或非线性回归模型。正如Sect。中所讨论的。如图2所示，通常情况是人们有兴趣使用一些与结构整体相关的属性作为结构标签，例如其内聚能，其介电常数等。在这些情况下，脊回归模型应该是使用的是基于结构之间的“全局”内核，  K(A,B) 而不是个别以原子为中心的环境之间的那些。这反映在如何组合环境之间的内核以提供适合表示本地环境与结构的整体属性之间的关系的内核。当目标属性可以看作是局部的，以原子为中心的贡献的加法组合，是最自然（和直接）的选择，这与方程式一致。（13），是
K(A,B)=∑j∈A,k∈BK(Xj,Xk). 
（56）
值得强调的是，在环境内核是基于SOAP描述符的线性内核的情况下，这个sum-kernel可以根据与整个结构相关的全局描述符来编写，
在新窗口中打开图像
（57）
我们介绍的地方
在新窗口中打开图像
（58）
在对称全局内核中组合来自各个环境的信息的另一种方法对应于平均每个环境的傅立叶系数，
在新窗口中打开图像
（59）
然后取得所得总和的哈尔积分。例如，对于ν  = 2，
在新窗口中打开图像
（60）
形式Eq。（56）更通用，并且可以容易地 在新窗口中 引入非线性内核，例如 Open图像， 对于该 窗口 ，描述符的显式表达将太麻烦。等式（56）还表明可以通过引入加权矩阵来推广环境核的组合
KW(A,B)=∑j∈A,k∈BWjk(A,B)K(Xj,Xk).
（61）
例如，可以确定结构中每个环境的重要性并设置。或者，可以使用来自最佳输运理论技术（Cuturi 2013）来定义熵正规化匹配（REMATCH）过程（德等人2016），其中w ^ JK是最相似的环境中的相匹配的双随机矩阵两种结构，忽略了非常不同的环境之间的环境核心  Wjk(A,B)=wj(A)wk(B) 
在新窗口中打开图像
（62）
其中d 2表示内核引起的平方距离Eq。（9）。参数γ对熵正则化进行加权，使得可以在最相似的环境对（γ  →0）的严格匹配与平均核之间进行插值，所述平均核均等地对所有对进行加权（γ  → ∞））。虽然这种结构使局部内核的组合相当复杂，但它提供了一种在结构之间的比较中引入非局部性元素的策略。考虑到计算REMatch内核的成本以及它阻止使用某些在单个环境中起作用的稀疏化策略的事实，当目标属性预期表现出非常强的非加性行为时，应该使用此方法，例如，当只有一个例如，当确定药物分子的活性时，系统的一部分涉及一个问题，REMatch已被证明可以显着改善ML模型的准确性（Bartók等人，2017）。
3.7多内核学习
我们已经证明SOAP描述符可以被看作是一类旋转对称的基于密度的描述符的一个可能的实施例，其也包含用于原子级机器学习的其他流行描述符，并且可以在很大程度上调整，例如，通过改变不同组件的加权方式。可以在同一形式中计算不同描述符的事实并不意味着它们是完全等价的：每个表达式或内核强调结构/属性关系的不同组件。例如，具有不同径向缩放或截止距离的内核将机器学习模型集中在短程，中程或长程交互上。然后自然地考虑是否可以通过组合与不同截止距离或不同身体顺序扩展水平相关联的描述符来构建更好的整体模型。这可以通过表单内核的加权组合来实现
Ktot(A,B)=∑ℵwℵKℵ(A,B), 
（63）
其中每个ķ ℵ对应于不同的模式。
这相当于一个属性的加性模型，类似于方程式中总能量的以原子为中心的分解的构造。（12）。在这种情况下，替代地，属性ÿ与每个结构有关被写成的贡献的总和与该各个内核相关联ķ ℵ  yℵ(A) 
y(A)=∑ℵyℵ(A)=∑ℵ,BxBwℵKℵ(A,B)
（64）
其中是每个代表结构的核心回归权重。权重瓦特ℵ对应于所估计的贡献，每个模型将给予最后一个属性，并且可以通过交叉验证或通过物理直觉来获得。例如，在多个径向截止的情况下，发现更小的权重应该与远程核相关联，这与远距离相互作用对总能量贡献很小（尽管通常是物理相关的）这一事实一致（Bartók等人，2017）。还应注意，假设对应于内核的描述符是线性无关的，则Eq。（  xB   B 63）有效地对应于增加维度的特征空间，通过连接与每个内核相关联的描述符（隐式或显式）而获得。

# 结论

我们已经建立了一个基于原子密度概念的数学框架，用于构建保持几何对称性和化学敏感极限的原子环境描述符。与核回归相结合，这允许在原子尺度上拟合复杂的物理性质模型，包括像原子间势（力场）和张量如多极矩和量子力学算子这样的标量。我们一般性地讨论如何扩展核回归以包括稀疏的参考结构选择以及预测和学习目标属性的线性函数。为了利用内核回归和量子力学之间的许多形式上的相似性，我们使用Dirac bra-ket表示法来制定关于SOAP描述符的主要结果。这种表示法还有助于明确SOAP描述符与其他流行的基于密度的方法之间的关系，以表示原子结构。可以通过许多不同的方式扩展和调整框架，以结合关于属性，结构和描述符之间关系的洞察。凭借物理原理，如对称性和相互作用的近视性，我们相信这种表述非常适合提供一个统一的框架来加工学习原子级属性。

**符号的含义**

A 
An item – structure, or atomic environment for which one wants to predict a property

K(A,B) 
The kernel function computed between items   A  and   B 

N
Number of input structures in the training set

M
Number of structures in the representative set

x
The vector of KRR weights, also written as xM; the weight associated with a structure   B  is indicated as   xB 

y
The vector containing the values of the target property, also written as yN.   yB  indicates the value for the item   B 

K
Kernel matrix

K MN
切片的核矩阵K，对应于集合M中的行和集合N中的列

Σ ķ
Sum-kernel，定义为一组配置中常规内核的总和

∇← 
导数运算符应用于核矩阵的第一个参数

∇→ 
导数运算符应用于核矩阵的第二个参数

K∇AB 
内核矩阵的导数，应用于它的第一个参数，关于结构中原子的坐标，结构作为其第二个参数  A   B 

在新窗口中打开图像
线性算子将观测值y与未观测到的原子能y '连接起来

在新窗口中打开图像
描述输入的抽象矢量  A 

在新窗口中打开图像
与和相关的功能之间的标量积。可以是显式标量乘积，也可以是等效于的抽象符号  A   B   K(A,B) 

g i  （r  ）
平滑函数 - 通常是高斯函数，用于表示与原子i相关的密度

在新窗口中打开图像
代表化学物种α的抽象矢量

[R
在3D笛卡尔坐标中定位。

[R
矢量r的模数

r^ 
单位向量r / r

[R 我
第i个原子的位置。

r ij
位移矢量r i  - r j在第i和第j个原子之间

ψ X Ĵ α（[R ）
物种α的原子密度以第j个原子为中心

Ylm(r^) 
第l，m次球谐

R n  （r）
第n个正交径向基函数

X j（ν）
阶数为ν的球面平均SOAP描述符

在新窗口中打开图像
阶数ν和非线性指数ζ的归一化SOAP核。

DMM' 升（[R ）
与旋转相关联的的Wigner旋转矩阵ř

αnα'n'l X Ĵ（2）
SOAPν = 2向量的径向/球形表示，对应于物种α和α'之间的功率谱

l 1 m 1 l 2 m 2 l m
一个Clebsch-Gordan系数

Ť λμ
对于张量T，λ的不可约球形分量的μ分量

X j，λμ（ν）
阶数为ν的λ- SOAP描述符，对应于以原子j为中心的不可约球面分量λμ

在新窗口中打开图像
由Hermitian运算符转换的内核  W^ 